In [ ]:
# TMO 2017-2018: practica-toxic.py
# Dpto. de C. de la Computación e I.A. (Univ. de Sevilla)
#=====================================================================

# ********************************************************************
# Nombre: Antonio
# Apellidos: Ramírez Hurtado
# ********************************************************************

# **************************** IMPORTANTE ****************************
# - Recordar escribir el nombre en la cabecera de este fichero.
# ********************************************************************

# ********************************************************************
# HONESTIDAD ACADÉMICA Y COPIAS: la realización de los ejercicios es
# un trabajo personal, por lo que deben completarse por cada
# estudiante de manera individual.  La discusión y el intercambio de
# información de carácter general con los compañeros se permite (e
# incluso se recomienda), pero NO AL NIVEL DE CÓDIGO. Igualmente el
# remitir código de terceros, obtenido a través de la red o cualquier
# otro medio, se considerará plagio.

# Cualquier plagio o compartición de código que se detecte significará
# automáticamente la calificación de CERO EN LA ASIGNATURA para TODOS
# los alumnos involucrados. 
# ********************************************************************

# Se pide crear un modelo de clasificación con alguno de los algoritmos
# vistos en clase e implementados en la librería scikit-learn, también se
# permite el uso de xgboost y keras (tensorflow). Se valorará
# el ajuste de parámetros realizado (aplicando validación cruzada), así como
# la transformaciones sobre los datos desarrolladas para mejorar el calidad
# (score) del modelo por validación cruzada (un buen ejemplo es el notebook
# Left que se puede encontrar en la enseñanza virtual)

# El conjunto de datos corresponde a comentarios realizados en wikipedia.
# Se pretende determinar si los comentarios tiene un carga negativa o no.
# que un anuncio dado tendrá para la comunidad de usuarios de este portal.
# Cada comentario puede ser clasificado como tóxico, muy tóxico, obsceno,
# insultante, con carga de odio y/o amenazante.

#    1. De los texto se pueden obtener atributos como la longitud, número 
#       de palabras, número de palabras únicas, número de mayúsculas, etc... 
#       También se pueden aplicar técnicas de vectorización de textos como 
#       CountVectorizer o TF-IDF entre otras. 
#    2. Se puede usar diferentes algoritmos: regresión lineal, naive bayes, 
#       random forest y despues intentar ensamblarlos (obtener la media de las
#       predicciones)

# Estas son algunas de las tranformaciones que se pueden realizar, pero no las
# únicas. Cualquier otra transformación que se lleve a cabo sobre los datos
# será tenida en cuenta positivamente.

# Se deberá entregar este archivo con las implementaciones realizadas y
# comentadas, el archivo de predicciones del mejor modelo encontrado y el score 
# asociado por cross-validación y el proporcionado por kaggle (enviar a 
# dsolis@us.es)

# Toda la información necesaria y los conjuntos de datos se pueden encontrar en:
# https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge

# *******************************************************************
# IMPORTANTE: El plazo de entrega es hasta la finalización de la 
# competición (20 de Febrero).
# ********************************************************************

# *******************************************************************
# IMPORTANTE: La competición está activa. Debéis subir las predicciones
# y que la plataforma os dé el score sobre el test. Podeis usar el nombre
# de usuario que considereis oportuno.
# ********************************************************************

# ********************************************************************
# IMPORTANTE: Se pueden consultar y usar los ejemplos de código y 
# transformaciones encontradas en: 
# https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/kernels
# ********************************************************************

# ********************************************************************
# IMPORTANTE: Para resolver cualquier duda contactar con David Solís 
# (dsolis@us.es)
# ********************************************************************



Cargamos las librerías necesarias

In [2]:
import pandas as pd
import numpy as np
from xgboost.sklearn import XGBClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
SEED = 5150

Cargamos los csv's en sendos dataframes.

In [3]:
train = pd.read_csv('train/train.csv')
test = pd.read_csv('test/test.csv')

Comprobamos que no existen elementos repetidos en ambos dataframes.

In [ ]:
set(train.id.values).intersection(test.id.values)

In [ ]:
set(train.id.values).intersection(test.id.values)

Seleccionamos nuestra variable regresora.

In [4]:
X = train['comment_text']

Seleccionamos nuestras variables objetivos.

In [ ]:
categories= [c for c in train.columns if not c in ['id', 'comment_text']]
y = train[categories]

Separaremos nuestro dataset X en dos conjuntos (entrenamiento y test), con el fin de comprobar la bondad de nuestro modelo clasificador.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=SEED)

## XGBoost

Será nuestro primer modelo clasificador. Antes de pasarle los datos al clasificador, aplicaremos algunas técnicas de vectorización de textos (CountVectorizer y TF-IDF) 

In [8]:
vect = CountVectorizer(max_df=0.95, min_df=2,max_features=300,stop_words=None)
tfidf = TfidfTransformer()
clf = XGBClassifier(base_estimator='gbtree', objetive='multi:softprob')

Definiremos una tubería para realizar las tres operaciones: vectorización del dataset, matriz de frecuencias y el modelo clasificador que hemos elegido para resolver el problema. 

In [ ]:
pipeline = Pipeline([('vect', vect), ('tfidf', tfidf), ('clf', clf)],)

Hemos definido una variable llamada params que nos ayudará a ajustar algunos de los parámetros que intervienen en el ajuste de la vectorización, la matriz de frecuencia y el modelo. En nuestro caso, ajustaremos el parámetro learning_rate del clasificador jugando con tres valores. 

In [ ]:
params = {
    'clf__learning_rate': [0.1, 0.05, 0.01],
    'clf__max_depth' : [6],
    'clf__silent' : [1],
    'clf__nthread' : [4]    
}

Para la optimización de los parámetros nos serviremos de GridSearchCV.

In [ ]:
model_training = GridSearchCV(pipeline, params, verbose=2, n_jobs=8, refit=True)

Como XGBoost no está preparado para trabajar con variables multietiquetas, tendremos que iterar sobre las categorías y ajustar el modelo para cada una de ellas.

In [ ]:
from sklearn.metrics import roc_auc_score 
for label in categories:
    model_training.fit(X_train, y_train[label])
    model_training.predict_proba(X_test)[:,1]
    best_parameters, score, _ = max(model_training.grid_scores_, key=lambda x: x[1])
    print('Raw AUC score:', score)
    for param_name in sorted(best_parameters.keys()):
        print("%s: %r" % (param_name, best_parameters[param_name]))

Sabiendo los mejores parámetros, podemos probar a mejorarlos.

In [ ]:
new_params = {
    'clf__learning_rate': [0.07, 0.06, 0.05],
    'clf__max_depth' : [6],
    'clf__silent' : [1],
    'clf__nthread' : [4],
    'clf__subsample': [0.8],
    'clf__colsample_bytree': [0.8],
}

Entrenamos de nuevo el modelo, esta vez con todos los datos disponibles en el dataset de entrenamiento. La validación cruzada será necesaria para comprobar si nuestro modelo es lo suficientemente generalista. 

In [ ]:
model = GridSearchCV(pipeline, new_params, n_jobs=8,
                      cv=StratifiedKFold(n_splits=5, shuffle=True, random_state= SEED), 
                      scoring='roc_auc',
                      verbose=2, refit=True)

Construiremos el dataset con las probabilidades de las predicciones para construir el fichero csv que enviaremos a Kaggle

In [ ]:
predictions = pd.DataFrame()
for label in categories:
    model.fit(X, y[label])
    pred = model.predict_proba(test["comment_text"])[:,1]
    predictions[label] = pd.Series(pred) 

In [ ]:
subm_XGB = pd.DataFrame(data=predictions, columns=categories)
subm_XGB = pd.concat((test, subm_XGB), axis=1)[['id'] + categories]
subm_XGB.to_csv("subm_XGB.csv", index=False)

#### Para Kaggle, el score de nuestro modelo XGBoost  es de 0.8617

## Random Forest

Random Forest permite modelar variables multietiqueta. Vamos a ajustar, con la ayuda de OneVsRestClassifier, un clasificador por clase. Para cada clasificador, la clase será ajustada frente a las otras clases.
Como en el modelo anterior, definiremos una tubería para aplicar técnicas de vectorización y definir el modelo.

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier

text_clf = Pipeline([('vect', CountVectorizer(max_features=500, max_df=0.95, min_df=2,
                                binary=False,
                                stop_words=None)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', OneVsRestClassifier(RandomForestClassifier(n_jobs=-1, n_estimators=20, random_state=SEED))),])

Definimos una batería de parámetros para ver con cuáles de ellos conseguimos un mejor ajuste.

In [ ]:
params = {
    'clf__estimator__max_features' : ['auto'], 
    'clf__estimator__min_samples_leaf': [1,2,3,4,5],
    'clf__estimator__criterion': ('gini','entropy')
}

Como en el modelo anterior, nos serviremos de una rejilla para pasarle los parámetros

In [ ]:
from sklearn.model_selection import GridSearchCV
model = GridSearchCV(text_clf, params, n_jobs=8, verbose=2, refit=True)

Ajustamos primero el modelo con los datos spliteados del dataset train

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred= model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

Obtenemos una precisión de 0.9063218063388975

Tomamos los mejores parámetros obtenidos en el ajuste anterior y modificamos algunos de ellos.

In [ ]:
learned_parameters = model.best_params_

In [ ]:
learned_parameters['clf__estimator__n_estimators'] = [100]
learned_parameters['clf__estimator__min_samples_leaf'] = [10]
learned_parameters['clf__estimator__criterion'] = ['entropy']
learned_parameters['clf__estimator__max_features'] = ['auto']
learned_parameters['clf__estimator__max_depth'] = [5]

Entrenamos de nuevo el modelo, esta vez con todos los datos disponibles en el dataset de entrenamiento. La validación cruzada será necesaria para comprobar si nuestro modelo es lo suficientemente generalista. 

In [ ]:
model = GridSearchCV(text_clf, learned_parameters, n_jobs=8,
                    cv = 5,
                    scoring='roc_auc',
                    verbose=2, refit=True)

In [ ]:
model.fit(X, y)

Obtenemos el nuevo score con todos los datos disponibles

In [ ]:
from sklearn.cross_validation import cross_val_score
scores = cross_val_score(model, X, y, scoring='roc_auc')

In [ ]:
np.mean(scores)

Nuestro score es de 0.9149130169770235

In [ ]:
pred = model.predict_proba(test['comment_text'])

In [ ]:
subm_RF = pd.DataFrame(data=pred, columns=categories)
subm_RF = pd.concat((test, subm_RF), axis=1)[['id'] + categories]
subm_RF.to_csv("subm_RF.csv", index=False)

#### Para Kaggle,  el score de nuestro modelo Random Forest es de 0.8939

# LSTM con Keras

LSTM es un tipo de red recurrente muy utilizada en la clasificación de textos

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.layers import SpatialDropout1D

In [ ]:
list_X = list(X.values)
list_test = list(test['comment_text'].values)

Realizamos un preprocesado de los datos convirtiendo cada palabra en un token

In [ ]:
max_words = 20000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(list_X + list_test)

La idea es tratar cada comentario como una secuencia de palabras. Las redes recurrentes tienen la ventaja de que cada salida depende no sólo de la entrada, si no de la salida anterior, por lo que la red conserva en memoria cada palabra que ha entrado en la red.

In [ ]:
X_sequences = tokenizer.texts_to_sequences(X.values)
test_sequences = tokenizer.texts_to_sequences(test['comment_text'].values)

La longitud máxima de los comentarios es maxlen = max(X.apply(len)), que en nuestro caso serían 5000; pero mi equipo era incapaz de crear secuencias tan largas 

In [ ]:
X_pad = pad_sequences(X_sequences, maxlen=3000)
test_pad = pad_sequences(test_sequences, maxlen=3000)

La red está compuesta por varias capas, cada una de ellas con una función determinada. La primera capa, llamada Embedding, se encargará de codificar la secuencias en vectores, de tal forma que las palabras cuyo significado sean parecidos tendrán vectores parecidos. La segunda capa es SpatialDropout, cuya función es la de evitar el sobreajuste del modelo; la tercera es la capa LSTM propiamente dicha; y la última, es la capa de salida (6 salidas, una por categoría)

In [ ]:
embed_dim = 32
lstm_out = 64
model = Sequential()
model.add(Embedding(max_words, embed_dim, input_length=3000))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(lstm_out, activation='tanh', recurrent_activation='hard_sigmoid', use_bias=True, dropout_U=0.2, dropout_W=0.2))
model.add(Dense(6,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics = ['accuracy'])
print(model.summary())

In [ ]:
batch_size = 32
model.fit(X_pad, y, epochs = 2, batch_size=batch_size, validation_split=0.1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/2
143613/143613 [==============================] - 38021s 265ms/step - loss: 0.2949 - acc: 0.9868 - val_loss: 0.2956 - val_acc: 0.9849
Epoch 2/2
143613/143613 [==============================] - 8722s 61ms/step - loss: 0.2850 - acc: 0.9800 - val_loss: 0.2881 - val_acc: 0.9821

Nos da un accuracy de 0.9868

Obtenemos las probabilidades para cada categoría y las guardamos en un archivo csv

In [ ]:
pred = model.predict(test_pad, batch_size=32, verbose=0)

In [ ]:
subm_LSTM = pd.DataFrame(data=pred, columns=categories)
subm_LSTM = pd.concat((test, subm_LSTM), axis=1)[['id'] + categories]
subm_LSTM.to_csv("subm_LSTM.csv", index=False)

#### Para Kaggle, el score de nuestro modelo de red recurrente es de 0.7375

Pese a la potencia de las redes recurrentes, una red mal diseñada se comporta mucho peor que cualquier otro modelo clásico.